# 02B - Neuro-Symbolic Compiler: Secondary Integration Notebook

This secondary notebook extends `02_neuro_symbolic_compiler.ipynb` with:
1. 1000x1000 scale matrix benchmarking
2. Grad-Shafranov (GS) equilibrium integration
3. End-to-end toy control-path latency (diagnostics -> physics -> compiler -> actuator)
4. Explicit deployment-scope boundaries

**Copyright clarity**
- Concepts: Copyright 1996-2026 Miroslav Sotek
- Code: Copyright 2024-2026 SCPN Fusion Core contributors
- License: GNU AGPL v3


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anulum/scpn-fusion-core/blob/main/examples/02_neuro_symbolic_compiler_secondary.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anulum/scpn-fusion-core/main?labpath=examples%2F02_neuro_symbolic_compiler_secondary.ipynb)

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scpn_fusion.scpn import StochasticPetriNet, FusionCompiler

## Step 1: Define a Plasma Control Petri Net

We model a simplified tokamak position controller with:
- 4 input places (sensor observations: R_high, R_low, Z_high, Z_low)
- 4 transitions (decision logic)
- 4 output places (actuator commands: PF_up, PF_down, PF_in, PF_out)

In [ ]:
net = StochasticPetriNet()

# Input places (sensor observations)
net.add_place("R_high", initial_tokens=0.0)
net.add_place("R_low",  initial_tokens=0.0)
net.add_place("Z_high", initial_tokens=0.0)
net.add_place("Z_low",  initial_tokens=0.0)

# Output places (actuator commands)
net.add_place("PF_up",   initial_tokens=0.0)
net.add_place("PF_down", initial_tokens=0.0)
net.add_place("PF_in",   initial_tokens=0.0)
net.add_place("PF_out",  initial_tokens=0.0)

# Transitions (control logic)
net.add_transition("T_correct_R_high", threshold=0.5)
net.add_transition("T_correct_R_low",  threshold=0.5)
net.add_transition("T_correct_Z_high", threshold=0.5)
net.add_transition("T_correct_Z_low",  threshold=0.5)

# Arcs: if R is too high → move plasma inward
net.add_arc("R_high", "T_correct_R_high", weight=1.0)
net.add_arc("T_correct_R_high", "PF_in", weight=1.0)

# If R is too low → move plasma outward
net.add_arc("R_low", "T_correct_R_low", weight=1.0)
net.add_arc("T_correct_R_low", "PF_out", weight=1.0)

# If Z is too high → push plasma down
net.add_arc("Z_high", "T_correct_Z_high", weight=1.0)
net.add_arc("T_correct_Z_high", "PF_down", weight=1.0)

# If Z is too low → push plasma up
net.add_arc("Z_low", "T_correct_Z_low", weight=1.0)
net.add_arc("T_correct_Z_low", "PF_up", weight=1.0)

net.compile()
print(net.summary())

## Step 2: Compile to Stochastic Neural Network

The compiler maps each transition to a stochastic LIF neuron.
If `sc-neurocore` is installed, it uses hardware-accurate bitstream
encoding. Otherwise, it falls back to NumPy float computation.

In [ ]:
compiler = FusionCompiler(bitstream_length=1024, seed=42)
compiled = compiler.compile(net)

print(f"Places:      {compiled.n_places}")
print(f"Transitions: {compiled.n_transitions}")
print(f"Stochastic:  {compiled.has_stochastic_path}")
print(f"Firing mode: {compiled.firing_mode}")
print()
print(compiled.summary())

## Step 3: Run Inference

We simulate a scenario where the plasma is displaced to R_high and Z_low.
The compiled network should activate PF_in (radial correction) and
PF_up (vertical correction).

In [ ]:
# Inject observation: plasma displaced R_high + Z_low
marking = np.zeros(compiled.n_places)
marking[0] = 0.8  # R_high active
marking[3] = 0.9  # Z_low active

W_in = compiled.W_in.toarray()
W_out = compiled.W_out.toarray()

print("Initial marking:", dict(zip(net.place_names, marking)))

# Step: compute transition firing
currents = W_in @ marking
fired = (currents >= compiled.thresholds).astype(float)
consumed = W_in.T @ fired
produced = W_out @ fired
new_marking = np.clip(marking - consumed + produced, 0.0, 1.0)

print("\nFired transitions:", dict(zip(net.transition_names, fired)))
print("\nNew marking:", dict(zip(net.place_names, new_marking)))
print("\n→ PF_in activated:", new_marking[6] > 0)   # PF_in
print("→ PF_up activated:", new_marking[4] > 0)   # PF_up

## Step 4: Multi-Step Evolution

Run the network for 30 steps with a time-varying disturbance signal.

In [ ]:
n_steps = 30
history = np.zeros((n_steps + 1, compiled.n_places))
marking = np.zeros(compiled.n_places)
history[0] = marking

for k in range(n_steps):
    # Time-varying disturbance
    t = k / n_steps
    marking[0] = 0.6 * np.sin(2 * np.pi * t) ** 2   # R_high oscillation
    marking[3] = 0.5 * np.cos(2 * np.pi * t) ** 2   # Z_low oscillation
    
    currents = W_in @ marking
    fired = (currents >= compiled.thresholds).astype(float)
    consumed = W_in.T @ fired
    produced = W_out @ fired
    marking = np.clip(marking - consumed + produced, 0.0, 1.0)
    history[k + 1] = marking

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
for i in range(4):
    axes[0].plot(history[:, i], label=net.place_names[i])
axes[0].set_ylabel("Input Places")
axes[0].legend(loc="upper right")
axes[0].set_title("Petri Net Token Evolution (30 steps)")

for i in range(4, 8):
    axes[1].plot(history[:, i], label=net.place_names[i])
axes[1].set_ylabel("Output Places")
axes[1].set_xlabel("Step")
axes[1].legend(loc="upper right")
plt.tight_layout()
plt.show()

## Step 5: Artifact Export / Import

The compiled network can be serialised as a JSON artifact for
deployment on embedded hardware or real-time controllers.

In [ ]:
import tempfile, os
from scpn_fusion.scpn import load_artifact, save_artifact

# Export
artifact = compiled.export_artifact(
    name="position_controller_v1",
    dt_control_s=0.001,
    readout_config={
        "action_specs": [
            {"place_idx": 4, "label": "PF_up"},
            {"place_idx": 5, "label": "PF_down"},
            {"place_idx": 6, "label": "PF_in"},
            {"place_idx": 7, "label": "PF_out"},
        ],
        "gains": [1000.0, 1000.0, 500.0, 500.0],
        "abs_max": [5000.0, 5000.0, 3000.0, 3000.0],
        "slew_per_s": [1e5, 1e5, 5e4, 5e4],
    },
    injection_config=[
        {"place_idx": 0, "label": "R_high"},
        {"place_idx": 1, "label": "R_low"},
        {"place_idx": 2, "label": "Z_high"},
        {"place_idx": 3, "label": "Z_low"},
    ],
)

fd, path = tempfile.mkstemp(suffix=".scpnctl.json")
os.close(fd)
save_artifact(artifact, path)
print(f"Saved artifact to: {path}")
print(f"File size: {os.path.getsize(path)} bytes")

# Reload
loaded = load_artifact(path)
print(f"\nReloaded: {loaded.meta['name']}")
print(f"Places: {loaded.nP}, Transitions: {loaded.nT}")
os.unlink(path)

## Performance Benchmarks

Timing the key computations in this notebook:
1. **Petri net compilation** (`FusionCompiler.compile`)
2. **Single inference step** (matrix multiply + threshold)
3. **30-step evolution loop** (multi-step token propagation)
4. **Artifact export/import round-trip**

In [ ]:
import timeit

# 1. Petri net compilation
def bench_compile():
    c = FusionCompiler(bitstream_length=1024, seed=42)
    c.compile(net)

t_compile = timeit.repeat(bench_compile, number=10, repeat=5)
print(f"FusionCompiler.compile (10 calls):")
print(f"  Mean: {np.mean(t_compile)*1000:.1f} ms +/- {np.std(t_compile)*1000:.1f} ms")
print(f"  Per call: {np.mean(t_compile)/10*1000:.2f} ms")

# 2. Single inference step (matrix multiply + threshold)
W_in_dense = compiled.W_in.toarray()
W_out_dense = compiled.W_out.toarray()
test_marking = np.zeros(compiled.n_places)
test_marking[0] = 0.8
test_marking[3] = 0.9

def bench_single_step():
    currents = W_in_dense @ test_marking
    fired = (currents >= compiled.thresholds).astype(float)
    consumed = W_in_dense.T @ fired
    produced = W_out_dense @ fired
    np.clip(test_marking - consumed + produced, 0.0, 1.0)

t_step = timeit.repeat(bench_single_step, number=10000, repeat=5)
print(f"\nSingle inference step (10000 calls):")
print(f"  Mean: {np.mean(t_step)*1000:.1f} ms +/- {np.std(t_step)*1000:.1f} ms")
print(f"  Per call: {np.mean(t_step)/10000*1e6:.2f} us")

# 3. 30-step evolution loop
def bench_evolution():
    m = np.zeros(compiled.n_places)
    for k in range(30):
        t = k / 30
        m[0] = 0.6 * np.sin(2 * np.pi * t) ** 2
        m[3] = 0.5 * np.cos(2 * np.pi * t) ** 2
        currents = W_in_dense @ m
        fired = (currents >= compiled.thresholds).astype(float)
        consumed = W_in_dense.T @ fired
        produced = W_out_dense @ fired
        m = np.clip(m - consumed + produced, 0.0, 1.0)

t_evo = timeit.repeat(bench_evolution, number=100, repeat=5)
print(f"\n30-step evolution (100 runs):")
print(f"  Mean: {np.mean(t_evo)*1000:.1f} ms +/- {np.std(t_evo)*1000:.1f} ms")
print(f"  Per run: {np.mean(t_evo)/100*1000:.2f} ms")

# 4. Artifact export round-trip
import tempfile, os

def bench_export_import():
    art = compiled.export_artifact(
        name="bench_test", dt_control_s=0.001,
        readout_config={"action_specs": [], "gains": [], "abs_max": [], "slew_per_s": []},
        injection_config=[],
    )
    fd, p = tempfile.mkstemp(suffix=".scpnctl.json")
    os.close(fd)
    save_artifact(art, p)
    load_artifact(p)
    os.unlink(p)

t_io = timeit.repeat(bench_export_import, number=10, repeat=3)
print(f"\nArtifact export/import round-trip (10 calls):")
print(f"  Mean: {np.mean(t_io)*1000:.1f} ms +/- {np.std(t_io)*1000:.1f} ms")
print(f"  Per call: {np.mean(t_io)/10*1000:.2f} ms")

## Summary

Baseline notebook flow is preserved above. The sections below add:
1. Scale benchmark at 1000x1000 matrix size
2. GS solver output coupling to diagnostic-derived controller inputs
3. Full toy-chain latency, not only neural net forward time


## Step 6: 1000x1000 Scale Benchmark

This benchmark reports dense 1000x1000 matrix performance for control-style linear algebra.
Results are hardware-dependent and should be interpreted as local-machine measurements.


In [ ]:
import time

rng_scale = np.random.default_rng(7)
N = 1000
W_scale = rng_scale.standard_normal((N, N), dtype=np.float64)
x_scale = rng_scale.standard_normal(N, dtype=np.float64)
A_scale = rng_scale.standard_normal((N, N), dtype=np.float64)
B_scale = rng_scale.standard_normal((N, N), dtype=np.float64)

def _bench_matvec(repeats=20):
    times = []
    _ = W_scale @ x_scale  # warmup
    for _ in range(repeats):
        t0 = time.perf_counter()
        _ = W_scale @ x_scale
        times.append((time.perf_counter() - t0) * 1e3)
    return np.asarray(times, dtype=np.float64)

def _bench_matmul(repeats=5):
    times = []
    _ = A_scale @ B_scale  # warmup
    for _ in range(repeats):
        t0 = time.perf_counter()
        _ = A_scale @ B_scale
        times.append((time.perf_counter() - t0) * 1e3)
    return np.asarray(times, dtype=np.float64)

matvec_ms = _bench_matvec()
matmul_ms = _bench_matmul()

print(f"Matrix size: {N}x{N}")
print(f"MatVec (W @ x) p50 / p95: {np.percentile(matvec_ms, 50):.3f} / {np.percentile(matvec_ms, 95):.3f} ms")
print(f"MatMul (A @ B) p50 / p95: {np.percentile(matmul_ms, 50):.3f} / {np.percentile(matmul_ms, 95):.3f} ms")


## Step 7: Physics Integration with GS Solver Output

The controller path below is connected to `FusionKernel.solve_equilibrium()` output.
We generate synthetic diagnostics from the solved equilibrium and use them as control inputs.


In [ ]:
import json
import os
import tempfile
from scpn_fusion.core import FusionKernel
from scpn_fusion.diagnostics.forward import generate_forward_channels


def _result_get(result, key, default=None):
    if isinstance(result, dict):
        return result.get(key, default)
    return getattr(result, key, default)


gs_config = {
    "reactor_name": "ITER-like",
    "grid_resolution": [65, 65],
    "dimensions": {"R_min": 1.0, "R_max": 9.0, "Z_min": -5.0, "Z_max": 5.0},
    "physics": {"plasma_current_target": 15.0, "vacuum_permeability": 1.2566370614e-6},
    "coils": [
        {"r": 3.5, "z": 4.0, "current": 5.0},
        {"r": 3.5, "z": -4.0, "current": 5.0},
        {"r": 9.0, "z": 4.0, "current": -3.0},
        {"r": 9.0, "z": -4.0, "current": -3.0},
        {"r": 6.2, "z": 5.5, "current": -1.5},
        {"r": 6.2, "z": -5.5, "current": -1.5},
    ],
    "solver": {
        "max_iterations": 80,
        "convergence_threshold": 1e-6,
        "relaxation_factor": 0.1,
        "solver_method": "multigrid",
    },
}

fd, gs_path = tempfile.mkstemp(suffix=".json")
os.close(fd)
with open(gs_path, "w", encoding="utf-8") as f:
    json.dump(gs_config, f)

kernel = FusionKernel(gs_path)
if hasattr(kernel, "initialize_grid"):
    kernel.initialize_grid()
if hasattr(kernel, "calculate_vacuum_field"):
    kernel.calculate_vacuum_field()

gs_result = kernel.solve_equilibrium()

psi_gs = np.asarray(kernel.Psi, dtype=np.float64).copy()
R = np.asarray(kernel.R, dtype=np.float64).copy()
Z = np.asarray(kernel.Z, dtype=np.float64).copy()
RR, ZZ = np.meshgrid(R, Z)

axis_r = _result_get(gs_result, "axis_r")
axis_z = _result_get(gs_result, "axis_z")
psi_axis = _result_get(gs_result, "psi_axis")
if axis_r is None or axis_z is None:
    axis_candidate = kernel.find_x_point(psi_gs)
    if isinstance(axis_candidate, tuple) and len(axis_candidate) == 3:
        axis_r, axis_z, psi_axis = axis_candidate
    else:
        (axis_r, axis_z), psi_axis = axis_candidate

axis_r = float(axis_r)
axis_z = float(axis_z)

gs_residual_value = float(_result_get(gs_result, "gs_residual", _result_get(gs_result, "residual", np.nan)))
if not np.isfinite(gs_residual_value):
    gs_residual_value = 1.0e-3

psi_min = float(np.min(psi_gs))
psi_max = float(np.max(psi_gs))
psi_norm = (psi_gs - psi_min) / max(psi_max - psi_min, 1e-12)
electron_density = 1.0e19 * (1.2 + 0.8 * (1.0 - psi_norm))
neutron_source = 1.0e17 * np.clip(1.1 - psi_norm, 0.0, None)
interferometer_chords = [
    ((float(R[0]), float(z0)), (float(R[-1]), float(z0)))
    for z0 in np.linspace(float(np.min(Z)) * 0.4, float(np.max(Z)) * 0.4, 8)
]
dV = float((R[1] - R[0]) * (Z[1] - Z[0]) * 2.0 * np.pi * np.mean(R))

forward_channels = generate_forward_channels(
    electron_density_m3=electron_density,
    neutron_source_m3_s=neutron_source,
    r_grid=R,
    z_grid=Z,
    interferometer_chords=interferometer_chords,
    volume_element_m3=dV,
)

print(f"GS converged: {bool(_result_get(gs_result, 'converged', False))}  iterations: {int(_result_get(gs_result, 'iterations', -1))}")
print(f"GS residual (used for pipeline): {gs_residual_value:.3e}")
print(f"Axis estimate: R={axis_r:.3f} m, Z={axis_z:.3f} m")
print(f"Interferometer channels: {forward_channels.interferometer_phase_rad.shape[0]}")
print(f"Neutron rate: {forward_channels.neutron_count_rate_hz:.3e} Hz")

fig, ax = plt.subplots(figsize=(6, 8))
levels = np.linspace(float(np.min(psi_gs)), float(np.max(psi_gs)), 30)
cs = ax.contour(RR, ZZ, psi_gs, levels=levels, cmap="viridis")
ax.plot(axis_r, axis_z, "rx", markersize=10, label="Axis estimate")
ax.set_xlabel("R [m]")
ax.set_ylabel("Z [m]")
ax.set_title("GS Equilibrium used for Diagnostics")
ax.set_aspect("equal")
ax.legend(loc="upper right")
plt.colorbar(cs, label="Psi [Wb/rad]")
plt.tight_layout()
plt.show()


## Step 8: End-to-End Toy Pipeline Latency

This section includes the full toy path per control tick:
1. Sensor preprocessing (diagnostics)
2. Physics model evaluation (equilibrium state features)
3. Neuro-symbolic forward pass
4. Actuator lag compensation

This is intentionally broader than NN-only forward timing.


In [ ]:
W_in_dense = compiled.W_in.toarray() if hasattr(compiled.W_in, "toarray") else np.asarray(compiled.W_in, dtype=np.float64)
W_out_dense = compiled.W_out.toarray() if hasattr(compiled.W_out, "toarray") else np.asarray(compiled.W_out, dtype=np.float64)
base_sensor = np.asarray(forward_channels.interferometer_phase_rad, dtype=np.float64)
if base_sensor.size < 8:
    base_sensor = np.pad(base_sensor, (0, 8 - base_sensor.size), mode="edge")


def preprocess_diagnostics(raw_sensor: np.ndarray) -> np.ndarray:
    x = np.asarray(raw_sensor, dtype=np.float64)
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    med = float(np.median(x))
    mad = float(np.median(np.abs(x - med)))
    scale = max(1.4826 * mad, 1e-9)
    return np.clip((x - med) / scale, -4.0, 4.0)


def evaluate_equilibrium_state(preprocessed: np.ndarray, gs_residual: float) -> tuple[float, float]:
    radial_metric = float(np.mean(preprocessed[:4]))
    vertical_metric = float(np.mean(preprocessed[4:8]))
    stability = float(np.clip(gs_residual / 1.0e-3, 0.0, 1.0))
    radial_error = float(np.tanh(radial_metric * (1.0 + 0.2 * stability)))
    vertical_error = float(np.tanh(vertical_metric * (1.0 + 0.2 * stability)))
    return radial_error, vertical_error


def build_marking_inputs(radial_error: float, vertical_error: float) -> np.ndarray:
    return np.array([
        np.clip(radial_error, 0.0, 1.0),
        np.clip(-radial_error, 0.0, 1.0),
        np.clip(vertical_error, 0.0, 1.0),
        np.clip(-vertical_error, 0.0, 1.0),
    ], dtype=np.float64)


def lag_compensate(desired: np.ndarray, prev_desired: np.ndarray, dt: float = 1e-3, tau: float = 4e-3) -> np.ndarray:
    return desired + (tau / max(dt, 1e-9)) * (desired - prev_desired)


def actuator_apply(command: np.ndarray, prev_applied: np.ndarray, dt: float = 1e-3, tau: float = 6e-3) -> np.ndarray:
    alpha = dt / (tau + dt)
    return prev_applied + alpha * (command - prev_applied)


gs_residual_for_chain = float(gs_residual_value) if "gs_residual_value" in globals() else 1.0e-3
if not np.isfinite(gs_residual_for_chain):
    gs_residual_for_chain = 1.0e-3

rng_chain = np.random.default_rng(123)
n_steps = 1000
marking = np.zeros(compiled.n_places, dtype=np.float64)
prev_desired = np.zeros(4, dtype=np.float64)
applied = np.zeros(4, dtype=np.float64)

lat_nn_ms = np.zeros(n_steps, dtype=np.float64)
lat_e2e_ms = np.zeros(n_steps, dtype=np.float64)
applied_trace = np.zeros((n_steps, 4), dtype=np.float64)

for k in range(n_steps):
    t_chain0 = time.perf_counter()
    disturbance = 0.03 * np.sin(2.0 * np.pi * k / 200.0)
    raw_sensor = base_sensor * (1.0 + disturbance) + rng_chain.normal(0.0, 1e-4, size=base_sensor.shape)
    pre = preprocess_diagnostics(raw_sensor)
    r_err, z_err = evaluate_equilibrium_state(pre, gs_residual_for_chain)

    marking[:] = 0.0
    marking[:4] = build_marking_inputs(r_err, z_err)

    t_nn0 = time.perf_counter()
    currents = W_in_dense @ marking
    fired = (currents >= compiled.thresholds).astype(np.float64)
    consumed = W_in_dense.T @ fired
    produced = W_out_dense @ fired
    new_marking = np.clip(marking - consumed + produced, 0.0, 1.0)
    desired = new_marking[4:8]
    t_nn1 = time.perf_counter()

    compensated = lag_compensate(desired, prev_desired)
    applied = actuator_apply(compensated, applied)
    prev_desired = desired
    applied_trace[k, :] = applied

    lat_nn_ms[k] = (t_nn1 - t_nn0) * 1e3
    lat_e2e_ms[k] = (time.perf_counter() - t_chain0) * 1e3

print("Toy pipeline latency over 1000 control ticks:")
print(f"  NN forward only p50/p95: {np.percentile(lat_nn_ms, 50):.4f} / {np.percentile(lat_nn_ms, 95):.4f} ms")
print(f"  End-to-end p50/p95:      {np.percentile(lat_e2e_ms, 50):.4f} / {np.percentile(lat_e2e_ms, 95):.4f} ms")
ratio = np.percentile(lat_e2e_ms, 95) / max(np.percentile(lat_nn_ms, 95), 1e-9)
print(f"  End-to-end / NN p95 ratio: {ratio:.2f}x")

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(lat_nn_ms, bins=40, alpha=0.7, label="NN forward")
axes[0].hist(lat_e2e_ms, bins=40, alpha=0.7, label="End-to-end")
axes[0].set_xlabel("Latency [ms]")
axes[0].set_ylabel("Count")
axes[0].set_title("Latency Distribution")
axes[0].legend()
axes[1].plot(applied_trace[:, 0], label="PF_up")
axes[1].plot(applied_trace[:, 1], label="PF_down")
axes[1].plot(applied_trace[:, 2], label="PF_in")
axes[1].plot(applied_trace[:, 3], label="PF_out")
axes[1].set_xlabel("Step")
axes[1].set_ylabel("Applied command")
axes[1].set_title("Lag-Compensated Actuator Trace")
axes[1].legend(loc="upper right", fontsize=8)
plt.tight_layout()
plt.show()


## Deployment Scope Clarification

This notebook now demonstrates both:
- Neural net forward-pass latency
- End-to-end toy latency (diagnostics -> equilibrium features -> neural net -> lag compensation)

Real deployment would still require:
- Sensor preprocessing and diagnostics integration on live streams
- Physics model evaluation against live equilibrium state
- Actuator lag compensation tuned to plant hardware
- Full DAQ/network/actuator timing budget verification

Copyright clarity:
- Concepts: Copyright 1996-2026
- Code: Copyright 2024-2026


In [ ]:
if "gs_path" in globals() and isinstance(gs_path, str) and os.path.exists(gs_path):
    os.unlink(gs_path)
    print(f"Cleaned up temporary config: {gs_path}")
